In [49]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
import numpy as np
import scipy.stats as st
import scipy.stats as stats

In [45]:
import aux_functions

In [121]:
df = pd.read_csv('df_clean_all.csv')
df.head()

,Unnamed: 0,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,...,testing_engagement,balance_level,tenure,Variation,visitor_id,visit_id,process_step,time,date,repetition
0,0,836976,6.0,73.0,60.5,U,2,45105.3,6.0,9.0,...,low,0-100k,5-10,Test,427070339_1413275162,228976764_46825473280_96584,confirm,11:51:13,2017-04-02,1
1,1,836976,6.0,73.0,60.5,U,2,45105.3,6.0,9.0,...,low,0-100k,5-10,Test,427070339_1413275162,228976764_46825473280_96584,confirm,11:47:50,2017-04-02,1
2,2,836976,6.0,73.0,60.5,U,2,45105.3,6.0,9.0,...,low,0-100k,5-10,Test,427070339_1413275162,228976764_46825473280_96584,confirm,11:46:45,2017-04-02,0
3,3,836976,6.0,73.0,60.5,U,2,45105.3,6.0,9.0,...,low,0-100k,5-10,Test,427070339_1413275162,228976764_46825473280_96584,step_3,11:23:08,2017-04-02,0
4,4,836976,6.0,73.0,60.5,U,2,45105.3,6.0,9.0,...,low,0-100k,5-10,Test,427070339_1413275162,228976764_46825473280_96584,step_2,11:22:24,2017-04-02,0


In [81]:
df_test = df[df['Variation'] == 'Test']
df_control = df[df['Variation'] == 'Control']
success_test = df_test[df_test['process_step'] == 'confirm'].shape[0]
success_control = df_control[df_control['process_step'] == 'confirm'].shape[0]

In [ ]:
# H₀ (hipótesis nula): No hay diferencia significativa en las tasas de finalización entre los dos grupos.
# H₁ (hipótesis alternativa): Hay una diferencia significativa en las tasas de finalización.

In [83]:
total_test = df_test.shape[0]
total_control = df_control.shape[0]

counts = [success_test, success_control]
totals = [total_test, total_control]

stat, p_value = proportions_ztest(counts, totals)

success_rate_test = success_test / total_test
success_rate_control = success_control / total_control

(stat, p_value, success_rate_test, success_rate_control)

(17.612218411058087,
 1.9851018964812982e-69,
 0.14489274856913173,
 0.12335388163608149)

In [ ]:
# H₀ (hipótesis nula): No hay diferencia significativa del 5% en las tasas de finalización.
# H₁ (hipótesis alternativa): Hay diferencia significativa del 5% en las tasas de finalización.

In [85]:
def verificar_mejora_umbral(success_rate_test, success_rate_control, umbral=0.05):
    aumento = success_rate_test - success_rate_control
    umbral_calculado = umbral * success_rate_control
    cumple_umbrales = aumento >= umbral_calculado

    return cumple_umbrales, aumento, umbral_calculado

In [87]:
cumple, aumento, umbral = verificar_mejora_umbral(success_rate_test, success_rate_control)
if cumple:
    print(f"El aumento en la tasa de finalización ({aumento:.4f}) supera el umbral del 5% ({umbral:.4f}). El nuevo diseño se justifica.")
else:
    print(f"El aumento en la tasa de finalización ({aumento:.4f}) no supera el umbral del 5% ({umbral:.4f}). El nuevo diseño no se justifica económicamente.")

El aumento en la tasa de finalización (0.0215) supera el umbral del 5% (0.0062). El nuevo diseño se justifica.


In [ ]:
# Hipótesis nula (H₀): La edad promedio de los clientes que interactúan con el nuevo proceso 
# es igual a la edad promedio de los que interactúan con el proceso anterior.
# Hipótesis alternativa (H₁): La edad promedio de los clientes que interactúan con el nuevo proceso 
# no es igual a la edad promedio de los que interactúan con el proceso anterior.

In [97]:
t_stat, p_value, resultado = comparar_edades(df_test, df_control, 'clnt_age')

print(f"Estadístico t: {t_stat}")
print(f"p-valor: {p_value}")
print(resultado)

Estadístico t: 7.833187064561318
p-valor: 4.771912120331317e-15
Rechazamos la hipótesis nula: las edades promedio son significativamente diferentes.


In [ ]:
# Hipótesis nula (H₀): Los grupos estan balanceados
# Hipótesis alternativa (H₁): Los grupos no estan balanceados

In [113]:
verificar_tamano_grupos(df)

Número de usuarios en el grupo Test: 176641
Número de usuarios en el grupo Control: 140482
Los grupos no están perfectamente balanceados. La diferencia es de 36159 usuarios.


(176641, 140482, 36159)

In [150]:
verificar_aleatorizacion(df, 'num_accts')

Estadístico t: -9.785061013095596
p-valor: 1.3155058968347921e-22
Hay diferencias significativas en la característica 'num_accts' entre los grupos.


(-9.785061013095596, 1.3155058968347921e-22)

In [152]:
verificar_aleatorizacion(df, 'clnt_age')

Estadístico t: 7.833187064561318
p-valor: 4.771912120331317e-15
Hay diferencias significativas en la característica 'clnt_age' entre los grupos.


(7.833187064561318, 4.771912120331317e-15)

In [154]:
verificar_aleatorizacion(df, 'bal')

Estadístico t: 2.879944198071027
p-valor: 0.003977722106826554
Hay diferencias significativas en la característica 'bal' entre los grupos.


(2.879944198071027, 0.003977722106826554)